<a href="https://colab.research.google.com/github/RiverGumSecurity/AILabs/blob/main/022_LargeLanguageModels/SFT_bhisblogs.ipynb" target="_new"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import warnings
import os
import sys
import pathlib
try:
    import unsloth
except:
    if 'google.colab' in sys.modules:
        !pip install unsloth

# suppresses some noisey warnings which are just annoying
warnings.filterwarnings('ignore')
max_seq_length = 4096

# Setup Hugging Face Credentials.
HF_APIKEY = ''
if 'google.colab' in sys.modules:
    from google.colab import userdata
    HF_APIKEY = userdata.get('HF_APIKEY')
else:
    with open(pathlib.Path.home() / '.hfkey') as hf:
        HF_APIKEY = hf.read().strip()
if not HF_APIKEY:
    print('[-] ERROR: Cannot continue without Hugging Face API Key')
    sys.exit(0)
os.environ['HF_TOKEN'] = HF_APIKEY

model, tokenizer = unsloth.FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = None, load_in_4bit = True
)

/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/bitsandbytes/cextension.py:48: SyntaxWarning: invalid escape sequence '\d'
  library_name = re.sub("cuda\d+", f"cuda{override_value}", library_name, count=1)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/unsloth/models/_utils.py:529: SyntaxWarning: invalid escape sequence '\.'
  source = re.sub("([^\.])nn\.", r"\1torch.nn.", source)
/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/unsloth/models/_utils.py:792: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/unsloth/models/_utils.py:890: SyntaxWarning: invalid escape sequence '\)'
  "self.rotary_emb = .+?\)", function,
/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/unsloth/models/llama.py:1515: SyntaxWarning: invalid escape sequence '\ '
  f"   \\\   /|    GPU: {gpu_stats.name}. Max memory: {max_memory} GB. Platform = {platform_system}.\n"\
/home/jsthyer/miniconda3/envs/llm/lib/python3.12/site-packages/unsloth/models/llama.py:1516: SyntaxWarning: invalid escape sequence '\_'
  f"O^O/ \_/ \\    Pytorch: {torch.__version__}. CUDA = {gpu_stats.major}.{

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
import re
import json
import datasets
import requests
from bs4 import BeautifulSoup

# get dataset
r = requests.get('https://raw.githubusercontent.com/RiverGumSecurity/Datasets/refs/heads/main/BHIS/bhis-blogs-cleaned.json')
data = json.loads(r.text)
ds = datasets.Dataset.from_list(data)

{'input': '',
 'instruction': 'You are an information security expert and have deep technical knowledge\nin the information security domain. You have the extensive knowledge of\nan experienced senior information security consultant.\nYou thoroughly understand a diverse amount of information security concepts including encryption, hashing,\nrisk analysis, penetration testing, red teaming, purple teaming,\nweb application testing, source code analysis, internal and external network testing.\n\nTake a step back and think step-by-step about how to achieve the best possible results.\nProvide a response to the input provided using one of the options below.\n\n\n<title>A Need For A Change - or - Burning Your Money</title>\n<taxonomies>InfoSec 201, News, breach, external breach, internal breach, it security, verizon breach report</taxonomies>\n<creation_date>Thu, 26 Mar 2015 17:43:47 +0000</creation_date>\n\n1. Summarize the blog post.\n2. Answer questions about the blog post.\n3. Provide deta

In [11]:
from transformers import TextStreamer

prompt = """
### Instruction:
Based on the following extract, generate five instruction-answer pairs. Each instruction
must ask to write about a specific topic contained in the context.
Each answer must provide a relevant paragraph based on the information found in the
context.
Only use concepts from the context to generate the instructions.
Instructions must never explicitly mention a context, a system, a course, or an extract.
Instructions must be self-contained and general.
Answers must imitate the writing style of the context.

Provide your response in JSON format with the following structure:

{{
"instruction_answer_pairs": [
        {{"instruction": "...", "answer": "..."}},
    ]
}}

### Input:
{}

### Response:
"""

unsloth.FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt.format(ds[4]['output'])], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4096)

<|begin_of_text|>
### Instruction:
Based on the following extract, generate five instruction-answer pairs. Each instruction
must ask to write about a specific topic contained in the context.
Each answer must provide a relevant paragraph based on the information found in the
context.
Only use concepts from the context to generate the instructions.
Instructions must never explicitly mention a context, a system, a course, or an extract.
Instructions must be self-contained and general.
Answers must imitate the writing style of the context.

Provide your response in JSON format with the following structure:

{
"instruction_answer_pairs": [
        {"instruction": "...", "answer": "..."},
    ]
}

### Input:
Joff Thyer When performing a penetration of test of organizations with Windows desktops many testers will now resort to using tools like Veil's Powershell Empire in order to inject shellcode directly into memory Without doubt this is a fantastic technique as it avoids writing to disk and